# Building a Custom Dense Layer

In this lab, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

> We will learn how to use initializers and also make the kernels of a simple dense layer and biases trainable so they can get updated in backpropogation step.
> For building any custom layer, we need to inherit it using `tensorflow.keras.layers.Layer` class.

In [1]:
# First we need to do some necessary imports
import tensorflow as tf
import numpy as np

## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [5]:
# Inherit from Layer class of keras API
from tensorflow.keras.layers import Layer

# Constructing a simple Dense layer
class SimpleDense(Layer):
    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super().__init__()
        # Create an object attribute for number of units
        self.units = units

    # Create a method to initialize kernels and biases 
    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # Create a weight initializer from a normal distribution
        w_init = tf.random_normal_initializer()
        # Create an object variable for weights
        self.w = tf.Variable(name="kernel", initial_value=w_init(shape=(input_shape[-1], self.units), dtype=tf.float32),
                             trainable=True)

        # Initialize the biases
        b_init = tf.zeros_initializer()
        # Create an object variable for biases
        self.b = tf.Variable(name="bias",
                            initial_value=b_init(shape=(self.units,), dtype=tf.float32), trainable=True)

    # Create call method for doing the computation in the layer'
    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

In [6]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense_2/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.00950447]], dtype=float32)>, <tf.Variable 'simple_dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [8]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float).reshape(-1, 1)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float).reshape(-1, 1)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

1/1 [==============================] - 0s 30ms/step
[[18.981522]]
[<tf.Variable 'simple_dense_4/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973216]], dtype=float32)>, <tf.Variable 'simple_dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99169624], dtype=float32)>]


- As we can see in the results of the previous cell, we can now print the values of weights and biases to check if our neural network is capturing the relation between x and y. Because the relation between x and y is **$y = 2x - 1$**, we need to check the weights and biases


In [16]:
weights = my_layer.w
biases = my_layer.b

print(f"The final and updated weights are: {weights.numpy().flatten()[0]}")
print(f"The final and updated biases are: {biases.numpy().flatten()[0]}")

The final and updated weights are: 1.997321605682373
The final and updated biases are: -0.9916962385177612


> As we can see the network has captured the underlying relation between $x, y$ pretty well!